In [1]:
import numpy as np
import heapq
import bisect
import math
from typing import Callable, TypeVar
from collections import defaultdict, Counter
import itertools
from intervaltree import IntervalTree
from functools import lru_cache
import graphviz  
import networkx as nx
import re
from shapely import Polygon
import z3

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

processors = {
  'int_list': lambda x: [int(y) for y in x.split()],
  'int_string': lambda x: [int(y) for y in x]
}

In [47]:
def day1():
    instructions = data(1, lambda x: (x[0], int(x[1:])))
    position, part1, part2 = 50, 0, 0
    for i, v in instructions:
        part2 += v//100
        v = v%100
        current = position + (v if i == 'R' else -v)
        if current >= 100 or (current <= 0 and position):
            part2 += 1
        if not (position := current%100):
            part1 += 1
    return part1, part2

day1()

(1034, 6166)

In [55]:
def day2():
    tree = IntervalTree()
    for x in data(2)[0].split(','):
        start, end = x.split('-')
        tree.addi(int(start), int(end)+1, x)

    cap, tested, part1 = len(str(tree.end()))//2, set(), 0
    for i in range(1, 10**cap):
        point = str(i)*2
        if tree.at(int(point)):
            part1 += int(point)
        tested.add(point)

    part2 = part1
    for i in range(1, 10**cap):
        digits = str(i)
        point = digits*2
        while len(point) < cap*2:
            point += digits
            if point in tested:
                continue
            if tree.at(int(point)):
                part2 += int(point)
            tested.add(point)
    
    return part1, part2

day2()

(40214376723, 50793864718)

In [52]:
def day3():
    jolts = data(3, processors['int_string'])

    def calculate_jolate(n):
        result = 0
        for jolt in jolts:
            index, switches = 0, ''
            for i in range(n, 0, -1):
                relevant = jolt[index:-i+1] or jolt[index:]
                value = max(relevant)
                idx = relevant.index(value)
                index += idx+1
                switches += str(value)
            result += int(switches)
        return result
    
    return calculate_jolate(2), calculate_jolate(12)

day3()

(17263, 170731717900423)

In [3]:
def day4():
    wall = np.pad(np.array(data(4, list)), 1)

    def remove_paper(wall, immediate=False):
        remove = set()
        for i, row in enumerate(np.lib.stride_tricks.sliding_window_view(wall, (3, 3))):
            for j, mask in enumerate(row):
                if mask[1, 1] == '@' and np.count_nonzero(mask=='@') <= 4:
                    if immediate:
                        wall[i+1, j+1] = '.'
                    remove.add((i+1, j+1))
        return remove

    to_remove = remove_paper(wall)
    part1, part2 = len(to_remove), 0
    while to_remove:
        for i, j in to_remove:
            wall[i, j] = '.'
        to_remove = remove_paper(wall, True)
        part2 += len(to_remove)

    return part1, part1+part2

day4()

(1491, 8722)

In [49]:
def day5():
    inventory = data(5)
    separator = inventory.index('')
    ranges = IntervalTree.from_tuples([
        (int(x.split('-')[0]), int(x.split('-')[-1])+1) for x in inventory[:separator]
    ])
    ranges.merge_overlaps()
    ingredients = [int(x) for x in inventory[separator+1:]]

    part1 = sum([bool(ranges.at(i)) for i in ingredients])
    part2 = sum([i.end-i.begin for i in ranges])
    return part1, part2

day5()

(775, 350684792662845)

In [128]:
def day6():
    operations = data(6, lambda x: x.split())
    part1 = sum([
        eval(expr[-1].join(expr[:-1]))
        for expr in zip(*operations)
    ])

    # Only works on solution
    # Does not work on example input
    raw = data(6, list)
    numbers = np.array(raw[:-1]).T
    ops = list(reversed(operations[-1]))
    current, part2 = [], 0
    for i in [''.join(col) for col in numbers] + [' ']:
        if i.strip():
            current.append(i)
        else:
            op = ops.pop()
            part2 += eval(op.join(current))
            current = []

    return part1, part2

day6()

(5060053676136, 9695042567249)

In [4]:
def day7():
    teleports = np.array(data(7, list))
    splits = defaultdict(lambda: [])
    for x in np.argwhere(teleports == '^'):
        heapq.heappush(splits[x[1]], (x[0]))

    splitted = set()
    visited, ends = {}, set()

    start = tuple(np.argwhere(teleports == 'S')[0])
    frontier = [(*start, start)]
    while frontier:
        y, x, source = heapq.heappop(frontier)
        if (y, x) in visited:
            visited[(y, x)] += visited[source]
            continue
        visited[(y, x)] = visited.get(source) or 1

        split = bisect.bisect_left(splits[x], y)
        if split < len(splits[x]):
            newy = splits[x][split]
            splitted.add((newy, x))
            heapq.heappush(frontier, (newy, x-1, (y, x)))
            heapq.heappush(frontier, (newy, x+1, (y, x)))
        else:
            ends.add((y, x))

    return len(splitted), sum(visited[i] for i in ends)

day7()

(1651, 108924003331749)

In [41]:
def day8(n=1000):
    junctions = data(8, lambda x: tuple([int(i) for i in x.split(',')]))

    G = nx.Graph()
    G.add_nodes_from(junctions)
    distances = []
    for a, b in itertools.combinations(G.nodes, 2):
        heapq.heappush(distances, (math.dist(a, b), a, b))

    i, a, b, ccs = 0, [0], [0], [0, 0, 0]
    while not nx.is_connected(G):
        if i == n:
            ccs = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
        d, a, b = heapq.heappop(distances)
        G.add_edge(a, b, length=d)
        i += 1

    return math.prod(ccs[:3]), a[0]*b[0]

day8()

(75680, 8995844880)

In [ ]:
def day9():
    tiles = data(9, lambda x: tuple([int(i) for i in x.split(',')]))
    polygon = Polygon(tiles)
    def area(a, b):
        l = abs(a[0]-b[0])+1
        w = abs(a[1]-b[1])+1
        return l*w

    distances = sorted([
        (area(a, b), a, b) for a, b in itertools.combinations(tiles, 2)
    ], reverse=True)

    for i in distances:
        d, a, b = i
        rectangle = Polygon([(a[0], a[1]), (a[0], b[1]), (b[0], b[1]), (b[0], a[1])])
        if polygon.contains(rectangle):
            return distances[0][0], d
    
day9()

(4782896435, 1540060480)

In [152]:
def day10():
    def parse_machine(line):
        items = line.split(' ')
        a, b, c = items[0], items[1:-1], items[-1]
        diagram = a.strip('[]')
        schematics = [[int(y) for y in x.split(',')] for x in re.findall(r'\((\d+(?:,\d+)*)\)', ' '.join(b))]
        requirements = [int(x) for x in c.strip('}{').split(',')]
        return diagram, schematics, requirements

    machines = data(10, parse_machine)

    def solve_machine(d, s, r, flag=False):
        solver = z3.Optimize()
        variables = [z3.Int(f'v{i}') for i in range(len(s))]
        for i in variables:
            solver.add(i >= 0)
        solver.minimize(sum(variables))

        for i, output in enumerate(r):
            f = sum([variables[j] for j, v in enumerate(s) if i in v])
            if flag:
                output = not d[i] == '.'
                f = sum([variables[j] for j, v in enumerate(s) if i in v])%2
            solver.add(output == f)

        solver.check()
        return solver.model()

    part1, part2 = 0, 0
    for machine in machines:
        indicator = solve_machine(*machine, True)
        joltage = solve_machine(*machine)
        part1 += sum([indicator[x].as_long() for x in indicator])
        part2 += sum([joltage[x].as_long() for x in joltage])
    return part1, part2

day10()

(512, 19857)

In [44]:
def day11_prep():
    G = nx.DiGraph()
    def parse_device(line):
        d, out = line.split(': ')
        for o in out.split(' '):
            G.add_edge(d, o)
        return d
    data(11, parse_device)

    # dot = graphviz.Digraph()
    # dot.node('dac', color='red')
    # dot.node('fft', color='red')
    # for edge in G.edges:
    #     dot.edge(edge[0], edge[1])
    # dot.render('data/day11.gv')

    return G

G = day11_prep()

In [45]:
# by visual inspection
bottlenecks = [
    ['svr'],
    ['qoq', 'ghp', 'ptu', 'dxw'],
    ['fft'],
    ['xni', 'uug', 'wgn'],
    ['xcb', 'qzo', 'deu', 'kpd', 'tbl'],
    ['yog', 'wsh', 'otb'],
    ['dac'],
    ['wfs', 'you', 'fpa'],
    ['out']
]

def day11(G):
    g = nx.DiGraph()
    for i in range(len(bottlenecks)-1):
        for a, b in itertools.product(bottlenecks[i], bottlenecks[i+1]):
            g.add_edge(a, b)

    @lru_cache()
    def shortest_path_length(a, b):
        return len(list(nx.shortest_simple_paths(G, a, b)))
    
    part1 = shortest_path_length('you', 'out')

    part2 = 0
    for path in list(nx.shortest_simple_paths(g, 'svr', 'out')):
        part2 += math.prod(shortest_path_length(*path[i:i+2]) for i in range(len(path)-1))

    return part1, part2

day11(G)

(753, 450854305019580)

In [ ]:
def day12():
    def parse_input():
        lines = data(12)[1:]

        def parse_block(b):
            nums = [int(x) for x in b.replace('x', ' ').replace(':', ' ').split()]
            return (nums[0], nums[1]), nums[2:]

        blocks, block = Counter(), 0
        for i, line in enumerate(lines):
            if 'x' in line:
                return [blocks[x] for x in blocks], [parse_block(l) for l in lines[i:]]
            elif not line:
                block += 1
                continue
            else:
                blocks[block] += line.count('#')

    blocks, boxes = parse_input()
    # blocks[-1] += 1 # Doesn't matter, but the other blocks can't fit into the middle for my input

    def check_box(dims, reqs):
        area = math.prod(dims)
        # min_bound = sum([math.prod(x) for x in zip(reqs, blocks)])
        max_bound = sum(reqs)*9
        return max_bound <= area

    return sum(check_box(*b) for b in boxes)

day12()

499